In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetRegressor
import torch

# Train verisi
train = pd.read_csv("../data/train.csv")

# datetime özellikleri
train["event_time"] = pd.to_datetime(train["event_time"])
train["hour"] = train["event_time"].dt.hour
train["dayofweek"] = train["event_time"].dt.dayofweek


In [ ]:
# %% [code]
def build_session_features_advanced(df, with_target=False):
    session_feats = df.groupby("user_session").agg(
        event_count=("event_type", "count"),
        unique_products=("product_id", "nunique"),
        unique_categories=("category_id", "nunique"),
        unique_event_types=("event_type", "nunique"),
        avg_hour=("hour", "mean"),
        dayofweek_mode=("dayofweek", lambda x: x.mode().iloc[0] if not x.mode().empty else -1),
    ).reset_index()

    df_time = df[["user_session", "event_time"]].copy()
    df_time["event_time"] = pd.to_datetime(df_time["event_time"])
    session_length = df_time.groupby("user_session").agg(
        session_length=("event_time", lambda x: (x.max() - x.min()).total_seconds())
    ).reset_index()
    session_feats = session_feats.merge(session_length, on="user_session", how="left")

    event_counts = df.groupby(["user_session", "event_type"]).size().unstack(fill_value=0).reset_index()
    session_feats = session_feats.merge(event_counts, on="user_session", how="left")

    for ev in ["ADD_CART", "VIEW", "REMOVE_CART", "PURCHASE"]:
        if ev in session_feats.columns:
            session_feats[f"ratio_{ev}"] = session_feats[ev] / session_feats["event_count"]

    session_feats["product_diversity"] = session_feats["unique_products"] / session_feats["event_count"]
    session_feats["category_diversity"] = session_feats["unique_categories"] / session_feats["event_count"]

    # Ek özellikler
    session_feats["session_length_log"] = np.log1p(session_feats["session_length"])
    session_feats["view_per_product"] = session_feats.get("VIEW", 0) / session_feats["unique_products"]
    session_feats["addcart_per_product"] = session_feats.get("ADD_CART", 0) / session_feats["unique_products"]
    session_feats["purchase_per_product"] = session_feats.get("PURCHASE", 0) / session_feats["unique_products"]
    session_feats["category_per_product"] = session_feats["unique_categories"] / session_feats["unique_products"]
    session_feats["events_per_unique_product"] = session_feats["event_count"] / session_feats["unique_products"]

    # Cyclical encoding
    session_feats["hour_sin"] = np.sin(2 * np.pi * session_feats["avg_hour"] / 24)
    session_feats["hour_cos"] = np.cos(2 * np.pi * session_feats["avg_hour"] / 24)
    session_feats["dayofweek_sin"] = np.sin(2 * np.pi * session_feats["dayofweek_mode"] / 7)
    session_feats["dayofweek_cos"] = np.cos(2 * np.pi * session_feats["dayofweek_mode"] / 7)

    # Numeric encoding
    for col in ["product_id", "category_id", "user_id"]:
        if df[col].dtype.name != "category":
            df[col] = df[col].astype("category")
        session_stats = df.groupby("user_session")[col].apply(lambda x: x.cat.codes.mean()).reset_index(name=f"{col}_mean_code")
        session_feats = session_feats.merge(session_stats, on="user_session", how="left")

    if with_target and "session_value" in df.columns:
        target = df.groupby("user_session")["session_value"].first().reset_index()
        session_feats = session_feats.merge(target, on="user_session", how="left")

    return session_feats


In [ ]:
train_features

In [10]:
train_features = build_session_features_advanced(train, with_target=True)

# Target ve feature ayır
X = train_features.drop(["user_session", "session_value"], axis=1)
y = train_features["session_value"]

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
X_train_np = X_train.values.astype(np.float32)
y_train_np = y_train.values.astype(np.float32).reshape(-1,1)
X_val_np = X_val.values.astype(np.float32)
y_val_np = y_val.values.astype(np.float32).reshape(-1,1)


In [19]:
import optuna
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
import joblib
from sklearn.metrics import mean_squared_error


def objective(trial):
    # Hiperparametreleri seç
    n_d = trial.suggest_int("n_d", 8, 64, step=8)
    n_a = trial.suggest_int("n_a", 8, 64, step=8)
    n_steps = trial.suggest_int("n_steps", 3, 10)
    gamma = trial.suggest_float("gamma", 1.0, 2.0, step=0.1)
    lr = trial.suggest_loguniform("lr", 1e-3, 1e-1)
    
    model = TabNetRegressor(
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        gamma=gamma,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=lr),
        mask_type="entmax"
    )
    
    model.fit(
        X_train_np, y_train_np,
        eval_set=[(X_val_np, y_val_np)],
        max_epochs=50,
        patience=5,
        batch_size=1024,
        virtual_batch_size=128
    )
    
    y_pred = model.predict(X_val_np)
    mse = mean_squared_error(y_val_np, y_pred)
    
    # En iyi modeli kaydet
    if trial.should_prune() is False:  # erken durdurma yoksa
        model_file = "best_tabnet_model.pkl"
        joblib.dump(model, model_file)
    
    return mse


In [20]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

print("Best RMSE:", study.best_value)
print("Best hyperparameters:", study.best_params)


[I 2025-08-29 17:13:53,620] A new study created in memory with name: no-name-1872b255-10e1-4ef3-8493-d176674768bf
C:\Users\hayka\AppData\Local\Temp\ipykernel_11476\2357511221.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-3, 1e-1)


epoch 0  | loss: 1790.78635| val_0_mse: 1044.01331|  0:00:03s
epoch 1  | loss: 649.67198| val_0_mse: 521.06427|  0:00:06s
epoch 2  | loss: 552.91056| val_0_mse: 479.4772|  0:00:10s
epoch 3  | loss: 508.01678| val_0_mse: 683.4433|  0:00:13s
epoch 4  | loss: 506.38059| val_0_mse: 564.72656|  0:00:16s
epoch 5  | loss: 491.78307| val_0_mse: 580.76422|  0:00:19s
epoch 6  | loss: 483.03268| val_0_mse: 544.99518|  0:00:23s
epoch 7  | loss: 480.13712| val_0_mse: 503.77475|  0:00:26s

Early stopping occurred at epoch 7 with best_epoch = 2 and best_val_0_mse = 479.4772


f:\Python\Python312\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-08-29 17:14:22,180] Trial 0 finished with value: 479.4772033691406 and parameters: {'n_d': 8, 'n_a': 24, 'n_steps': 4, 'gamma': 2.0, 'lr': 0.030751288603550666}. Best is trial 0 with value: 479.4772033691406.
C:\Users\hayka\AppData\Local\Temp\ipykernel_11476\2357511221.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-3, 1e-1)
f:\Python\Python312\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 2901.95853| val_0_mse: 2120.11279|  0:00:05s
epoch 1  | loss: 1276.46697| val_0_mse: 870.17035|  0:00:10s
epoch 2  | loss: 689.76253| val_0_mse: 666.56421|  0:00:15s
epoch 3  | loss: 627.84015| val_0_mse: 655.91351|  0:00:20s
epoch 4  | loss: 562.72998| val_0_mse: 573.65747|  0:00:26s
epoch 5  | loss: 523.99524| val_0_mse: 632.66846|  0:00:31s
epoch 6  | loss: 512.86494| val_0_mse: 541.03943|  0:00:37s
epoch 7  | loss: 509.70938| val_0_mse: 532.81689|  0:00:42s
epoch 8  | loss: 505.06284| val_0_mse: 534.38971|  0:00:48s
epoch 9  | loss: 463.88042| val_0_mse: 518.29297|  0:00:53s
epoch 10 | loss: 444.86708| val_0_mse: 497.47159|  0:00:58s
epoch 11 | loss: 434.53926| val_0_mse: 526.62292|  0:01:04s
epoch 12 | loss: 429.61026| val_0_mse: 558.49792|  0:01:09s
epoch 13 | loss: 427.66411| val_0_mse: 492.32916|  0:01:14s
epoch 14 | loss: 404.89076| val_0_mse: 469.72089|  0:01:20s
epoch 15 | loss: 416.18099| val_0_mse: 420.60779|  0:01:25s
epoch 16 | loss: 400.65414| val_0_mse

f:\Python\Python312\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-08-29 17:16:17,555] Trial 1 finished with value: 420.6077880859375 and parameters: {'n_d': 48, 'n_a': 64, 'n_steps': 4, 'gamma': 1.9, 'lr': 0.0029818584804036387}. Best is trial 1 with value: 420.6077880859375.
C:\Users\hayka\AppData\Local\Temp\ipykernel_11476\2357511221.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-3, 1e-1)
f:\Python\Python312\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


...

epoch 0  | loss: 892.31883| val_0_mse: 1274.37329|  0:00:08s
epoch 1  | loss: 580.95369| val_0_mse: 631.85681|  0:00:16s
epoch 2  | loss: 538.21748| val_0_mse: 610.10193|  0:00:25s
epoch 3  | loss: 585.16131| val_0_mse: 609.31921|  0:00:34s
epoch 4  | loss: 575.28207| val_0_mse: 531.09082|  0:00:42s
epoch 5  | loss: 555.55892| val_0_mse: 553.24854|  0:00:51s
epoch 6  | loss: 526.27635| val_0_mse: 502.36099|  0:00:59s
epoch 7  | loss: 508.05677| val_0_mse: 548.49426|  0:01:08s
epoch 8  | loss: 487.58393| val_0_mse: 519.9632|  0:01:16s
epoch 9  | loss: 491.5156| val_0_mse: 509.70877|  0:01:24s
epoch 10 | loss: 520.16899| val_0_mse: 591.38385|  0:01:33s
epoch 11 | loss: 493.51304| val_0_mse: 479.21945|  0:01:41s
epoch 12 | loss: 500.66278| val_0_mse: 479.50894|  0:01:50s
epoch 13 | loss: 459.16815| val_0_mse: 466.34528|  0:01:59s
epoch 14 | loss: 468.53426| val_0_mse: 495.14215|  0:02:07s
epoch 15 | loss: 462.01216| val_0_mse: 488.89127|  0:02:16s
epoch 16 | loss: 456.43572| val_0_mse: 52

f:\Python\Python312\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-08-29 17:49:00,869] Trial 14 finished with value: 466.34527587890625 and parameters: {'n_d': 32, 'n_a': 48, 'n_steps': 8, 'gamma': 1.1, 'lr': 0.08205226208593713}. Best is trial 5 with value: 355.4302978515625.
C:\Users\hayka\AppData\Local\Temp\ipykernel_11476\2357511221.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-3, 1e-1)
f:\Python\Python312\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 3578.84203| val_0_mse: 3403.96729|  0:00:07s
epoch 1  | loss: 3012.77328| val_0_mse: 3037.44678|  0:00:14s
epoch 2  | loss: 2571.23486| val_0_mse: 2659.6748|  0:00:22s
epoch 3  | loss: 2211.8852| val_0_mse: 2233.02979|  0:00:31s
epoch 4  | loss: 1922.14314| val_0_mse: 1923.6792|  0:00:40s
epoch 5  | loss: 1641.06424| val_0_mse: 1774.36646|  0:00:48s
epoch 6  | loss: 1416.66305| val_0_mse: 1462.95081|  0:00:56s


[W 2025-08-29 17:50:03,278] Trial 15 failed with parameters: {'n_d': 48, 'n_a': 32, 'n_steps': 7, 'gamma': 1.5, 'lr': 0.001059609576931035} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "f:\Python\Python312\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\hayka\AppData\Local\Temp\ipykernel_11476\2357511221.py", line 26, in objective
    model.fit(
  File "f:\Python\Python312\Lib\site-packages\pytorch_tabnet\abstract_model.py", line 258, in fit
    self._train_epoch(train_dataloader)
  File "f:\Python\Python312\Lib\site-packages\pytorch_tabnet\abstract_model.py", line 489, in _train_epoch
    batch_logs = self._train_batch(X, y)
                 ^^^^^^^^^^^^^^^^^^^^^^^
  File "f:\Python\Python312\Lib\site-packages\pytorch_tabnet\abstract_model.py", line 527, in _train_batch
    output, M_loss = self.network(X)
                     ^^^

KeyboardInterrupt: 

In [ ]:
import joblib
from sklearn.metrics import mean_squared_error
import numpy as np

# Kaydedilen en iyi modeli yükle
best_model = joblib.load("best_tabnet_model.pkl")

# Validation seti üzerinde tahmin
y_pred_val_loaded = best_model.predict(X_val_np)

# RMSE hesapla
mse_loaded = mean_squared_error(y_val_np, y_pred_val_loaded)
rmse_loaded = np.sqrt(mse_loaded)

print(f"Loaded model Validation MSE: {mse_loaded:.4f}")
print(f"Loaded model Validation RMSE: {rmse_loaded:.4f}")


Loaded model Validation MSE: 466.3453
Loaded model Validation RMSE: 21.5950


In [24]:
# Test verisini yükleme
test = pd.read_csv("../data/test.csv")

# Genel bilgi
print(test.info())
# Test verisine datetime özelliklerini ekleyelim
test["event_time"] = pd.to_datetime(test["event_time"])
test["hour"] = test["event_time"].dt.hour
test["dayofweek"] = test["event_time"].dt.dayofweek

# İlk birkaç satır
display(test.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62951 entries, 0 to 62950
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   event_time    62951 non-null  object
 1   event_type    62951 non-null  object
 2   product_id    62951 non-null  object
 3   category_id   62951 non-null  object
 4   user_id       62951 non-null  object
 5   user_session  62951 non-null  object
dtypes: object(6)
memory usage: 2.9+ MB
None


,event_time,event_type,product_id,category_id,user_id,user_session,hour,dayofweek
0,2025-06-28 10:09:58+00:00,ADD_CART,PROD_015000,CAT_00019,USER_109759,SESSION_164059,10,5
1,2025-06-25 11:57:50+00:00,ADD_CART,PROD_023887,CAT_00010,USER_010614,SESSION_109583,11,2
2,2025-06-30 14:34:20+00:00,ADD_CART,PROD_022673,CAT_00090,USER_041338,SESSION_171382,14,0
3,2025-06-30 22:12:18+00:00,ADD_CART,PROD_004664,CAT_00280,USER_015376,SESSION_137110,22,0
4,2025-06-26 16:55:18+00:00,ADD_CART,PROD_027815,CAT_00027,USER_054449,SESSION_146503,16,3


In [33]:
# %% [code]
def build_session_features_test_rowlevel(df):
    # --- Oturum bazlı özetleri hesapla ---
    session_feats = df.groupby("user_session").agg(
        event_count=("event_type", "count"),
        unique_products=("product_id", "nunique"),
        unique_categories=("category_id", "nunique"),
        unique_event_types=("event_type", "nunique"),
        avg_hour=("hour", "mean"),
        dayofweek_mode=("dayofweek", lambda x: x.mode().iloc[0] if not x.mode().empty else -1),
    ).reset_index()

    # --- Session length ---
    df_time = df[["user_session", "event_time"]].copy()
    df_time["event_time"] = pd.to_datetime(df_time["event_time"])
    session_length = df_time.groupby("user_session").agg(
        session_length=("event_time", lambda x: (x.max() - x.min()).total_seconds())
    ).reset_index()
    session_feats = session_feats.merge(session_length, on="user_session", how="left")

    # --- Event type counts ---
    event_counts = df.groupby(["user_session", "event_type"]).size().unstack(fill_value=0).reset_index()
    session_feats = session_feats.merge(event_counts, on="user_session", how="left")

    # --- Ratios ve ek özellikler ---
    for ev in ["ADD_CART", "VIEW", "REMOVE_CART", "PURCHASE"]:
        if ev in session_feats.columns:
            session_feats[f"ratio_{ev}"] = session_feats[ev] / session_feats["event_count"]

    session_feats["product_diversity"] = session_feats["unique_products"] / session_feats["event_count"]
    session_feats["category_diversity"] = session_feats["unique_categories"] / session_feats["event_count"]
    session_feats["events_per_unique_product"] = session_feats["event_count"] / session_feats["unique_products"]
    session_feats["view_per_product"] = session_feats.get("VIEW", 0) / session_feats["unique_products"]
    session_feats["addcart_per_product"] = session_feats.get("ADD_CART", 0) / session_feats["unique_products"]
    session_feats["purchase_per_product"] = session_feats.get("PURCHASE", 0) / session_feats["unique_products"]
    session_feats["category_per_product"] = session_feats["unique_categories"] / session_feats["unique_products"]
    session_feats["session_length_log"] = np.log1p(session_feats["session_length"])

    # --- Cyclical encoding ---
    session_feats["hour_sin"] = np.sin(2 * np.pi * session_feats["avg_hour"] / 24)
    session_feats["hour_cos"] = np.cos(2 * np.pi * session_feats["avg_hour"] / 24)
    session_feats["dayofweek_sin"] = np.sin(2 * np.pi * session_feats["dayofweek_mode"] / 7)
    session_feats["dayofweek_cos"] = np.cos(2 * np.pi * session_feats["dayofweek_mode"] / 7)

    # --- Numeric encoding ---
    for col in ["product_id", "category_id", "user_id"]:
        if df[col].dtype.name != "category":
            df[col] = df[col].astype("category")
        session_stats = df.groupby("user_session")[col].apply(lambda x: x.cat.codes.mean()).reset_index(name=f"{col}_mean_code")
        session_feats = session_feats.merge(session_stats, on="user_session", how="left")

    # --- Oturum özetlerini her satıra kopyala ---
    df_rowlevel = df.merge(session_feats, on="user_session", how="left")
    
    return df_rowlevel


In [ ]:
y_pred_test

In [43]:
# %% [code]
# 1️⃣ Test dataframe’ini row-level ve oturum özetleriyle hazırla
test_rowlevel = build_session_features_test_rowlevel(test)

# X_train: eğitim sırasında kullanılan dataframe
# test_rowlevel: test dataframe row-level feature’ları

# 1️⃣ Feature hizalama
X_test_aligned = test_rowlevel[X_train.columns.intersection(test_rowlevel.columns)].copy()

# 2️⃣ Eksik kolonları sıfır ile ekle
for col in X_train.columns:
    if col not in X_test_aligned.columns:
        X_test_aligned[col] = 0

# 3️⃣ Sütun sırasını eğitimle aynı yap
X_test_aligned = X_test_aligned[X_train.columns]

# 4️⃣ NumPy array'e çevir
X_test_np = X_test_aligned.values.astype(np.float32)


# %% [code]
# 3️⃣ TabNet ile tahmin
y_pred_test = best_model.predict(X_test_np).flatten()

test['pred'] = y_pred_test
y_pred_session = test.groupby('user_session')['pred'].mean().reset_index()


# 4️⃣ Submission dosyasını oluştur ve kaydet
submission = y_pred_session.rename(columns={'pred': 'session_value'})
submission.to_csv("submission.csv", index=False)

print("Submission kaydedildi: submission.csv")
submission.head()


Submission kaydedildi: submission.csv


,user_session,session_value
0,SESSION_000000,264.517395
1,SESSION_000013,16.970219
2,SESSION_000022,40.910393
3,SESSION_000024,23.617821
4,SESSION_000025,35.236328
